<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">


# Web Scraping for Indeed.com and Predicting Salaries

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

# Web scraping test

In [27]:
#setting URL and using beautiful soup to parse scraped text
url = "https://www.indeed.co.uk/jobs?q=Data+Scientist&l=London&radius=0"

r=requests.get(url)
r.text[:1000]
soup = BeautifulSoup(r.text, 'html.parser')

In [28]:
#example of first job
test=soup.find_all('div', class_ = 'result')
print(test[0].prettify())

<div class="jobsearch-SerpJobCard unifiedRow row result" data-ci="363252621" data-empn="5940578736440115" data-jk="c4733c6580c56872" id="pj_c4733c6580c56872">
 <style>
  .jobcard_logo{margin:6px 0}.jobcard_logo img{width:auto;max-width:80px;max-height:30px}.jasxrefreshcombotst .jobcard_logo img{max-height:2rem;max-width:100%}
 </style>
 <h2 class="title">
  <a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0ARyhN0hxCsk44jc7iGPq5C3PdshFfDMnYbN1CoUR1yamM7iHaFLyF9J5ymqkdHAZUmLF7aP5cm4nrptxZ8BwA-AVQQFT-REZFy6SRMz9ZsoY5pfhP2nx_lSGBJigqiDDEdHmdevhWCN7SCiWLH-0n88avIG7_VJ2ES13C9dFW5Z3DZbS-ngHdlxNxzZ0faVlO5OxrlncGiW3uWEHrRivDR3akxzuD8xw6xwzq_SSWM6sUSGt7FQZRvvxZfDuDZRSjKQPBdJC9i9t6lro5vTNKe4lKjSR9uHd08qfxm6cvkpIYuSp9Jtfkk45bpYc7SgzYBE4Kasb_R7X0mb9yj28k3NGBSVP9JutzOOzzJ_X3p1tg6zfJgw-UXO-lSx0jEiFmzsl5lBzUj6T01cFB0oUyhZDlGTy0vTK9QQ448ZGTDwR5iUVoarWy53qCigDPJNEilhdWNZj9sJubI3TQhb4Mc8aikEvuVw-w=&amp;p=0&amp;fvj=1&amp;vjs=3" id="sja0" onclick="setRefin

### Defining functions to extract location, company, salary and job title

In [11]:
## LOCATION FUNCTION

def extract_location_from_result(job):
        try:
            return job.find('span', attrs={'class' : 'location'}).text.strip()
        except:
            try:
                return job.find('div', attrs={'class' : 'location'}).text.strip()
            except:
                return np.nan

In [12]:
## COMPANY FUNCTION

def extract_company_from_result(job):
        try:
            return job.find('span', attrs={'class' : 'company'}).text.strip()
        except:
            return np.nan

In [13]:
## SALARY FUNCTION

def extract_salary_from_result(job):
        try:
            return job.find('span', attrs={'class' : 'salaryText'}).text.strip()
        except:
            return np.nan

In [14]:
## JOB FUNCTION

def extract_title_from_result(job):
        try:
            return job.find('h2', attrs={'class' : 'title'}).text.strip()
        except:
            return np.nan

### Testing functions on test result

In [21]:
title_non=[]
company_non=[]
location_non=[]
salary_non=[]

for i in test:
            title_non.append(extract_title_from_result(i))
            company_non.append(extract_company_from_result(i))
            location_non.append(extract_location_from_result(i))
            salary_non.append(extract_salary_from_result(i))
            
            
df = pd.DataFrame({'company': company_non,
                   'location': location_non,
                   'title': title_non,
                   'salary': salary_non})

df

,company,location,title,salary
0,Funding Circle UK,London EC4V,Data Scientist - Customer Analytics,NaN
1,Paddy Power Betfair,London,Data Scientist\nnew,NaN
2,Funding Circle UK,London EC4V,Data Scientist,NaN
3,Intelligent Data Technologies Limited (Lucid),London,Neo4j / Tiger Graph Data Scientist,"£35,000 - £55,000 a year"
4,Hastings Direct,London,Junior Data Scientist\nnew,NaN
5,G-Research,London,Data Scientist,NaN
6,Novafutur,London,Junior Data Scientist,NaN
7,"JPMorgan Chase Bank, N.A.",London,Applied Data Scientist - Athena Applied Intell...,NaN
8,EDF Energy Limited,London,Data Scientist\nnew,NaN
9,UK Government - Government Actuary's Department,London EC4A 1AB,Lead Data Scientist,"£56,480 - £80,764 a year"


In [29]:
#Applying the above to larger scale.
#Set search terms, number of jobs to search per city, list of cities.

import time, random

url_template_non = "https://www.indeed.co.uk/jobs?q=%28Data+or+Scientist+or+Data+or+Analyst+or+Machine+or+Learning%29&l={}&limit=50&radius=25&start={}"
max_results_per_city = 50

cities_non = ['Birmingham', 'Sheffield', 'Manchester', 'Cambridge', 'Oxford', 'Reading', 
          'Bath', 'Liverpool', 'Leeds', 'Edinburgh', 'Glasgow', 'Cardiff', 'Nottingham', 'Newcastle']

job_cards_non = []
company_non=[]
location_non=[]
salary_non=[]
title_non=[]
search_city_non=[]

for city in cities_non:
    for start in range(0, max_results_per_city, 10):
            try:
                r=requests.get(url_template_non.format(city, start)) #extract job card and then each variable
            except:
                print('page not found')
            
            search_city_non.append(city)
            
            url_soup = BeautifulSoup(r.text, 'html.parser')
            
            cards = url_soup.find_all('div', attrs={'class' : 'jobsearch-SerpJobCard'})
            
            job_cards_non.append(cards) #is a list of lists.

            time.sleep(random.randint(3, 6))


for job in job_cards_non:
    for i in job:
            title_non.append(extract_title_from_result(i))
            company_non.append(extract_company_from_result(i))
            location_non.append(extract_location_from_result(i))
            salary_non.append(extract_salary_from_result(i))

In [31]:
df2 = pd.DataFrame({'company': company_non,
                   'location': location_non,
                   'title': title_non,
                   'salary': salary_non})

df2

,company,location,title,salary
0,Serco Plc,Birmingham,Data Analyst\nnew,"£40,000 a year"
1,H&M,Birmingham,Full Time Sales Advisor - Birmingham Newstreet...,NaN
2,BT,Birmingham,Data Scientist,NaN
3,Skillcert,Birmingham,Trainee E-sport/ Gaming Operations Assistant\nnew,NaN
4,Public Health England,Birmingham,919-GC-17950615-EXT Public Health Intelligence...,"£28,588 - £35,612 a year"
...,...,...,...,...
711,DXC Technology,Newcastle upon Tyne,Mainframe Analyst,NaN
712,Just Eat,North East,Planning Analyst - Just Eat Takeaway.com\nnew,"£34,000 a year"
713,Mediaworks,Gateshead,Data Analyst,NaN
714,IQVIA,Newcastle upon Tyne,Covid-19 Study Worker\nnew,NaN


### Creating dataframe/write df to csv


In [ ]:
#first job only.
job_cards[0][0]

In [54]:
df2.to_csv('pathname.csv')